# Predict AI Model Runtime (Tile Only)

## Prerequisites & Setup

 - [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/stable/) (aka PyG) is used for the majority a graph-based neural network tasks.
 - [pytorch_scatter](https://github.com/rusty1s/pytorch_scatter) is required for the `global_max_pool` function used in the [model](#The-Model) to function efficiently. Note: This will not compile on Kaggle, so it has been removed.

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import multiprocessing
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.init as init
from IPython.display import display, HTML
from pathlib import Path
from tqdm.auto import tqdm
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import OneCycleLR
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn.models import MLP, GraphSAGE
from torch_geometric.nn.pool import global_mean_pool, global_max_pool
from sklearn.preprocessing import MinMaxScaler
from tqdm.contrib.concurrent import process_map

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Import Data

In [ ]:
# base_path = Path('/kaggle/input')
# path = Path(base_path/'predict-ai-model-runtime/npz_all/npz/tile/xla')

path = Path('npz/tile/xla')

Fetch each model file as needed and concatenate them into a dictionary of DataFrames according to each split. This is done via `multiprocessing` using `process_map` from `tdqm`. It offers a noticable speed increase due to the I/O limits introduced by `np.load`.

In [ ]:
def process_split(file):
    d = dict(np.load(file))
    d['file'] = file.stem
    return d

def load_df(path):
    splits = ['train', 'valid', 'test']
    cpu_count = multiprocessing.cpu_count()
    df_dict = {}
    for i, split in enumerate(splits, start=1):
        split_path = path/split
        files = list(split_path.glob('*.npz'))
        df_list = []
        pbar = process_map(process_split, files, unit='file',
                           desc=f'({i}/{len(splits)}) {split}, Processing',
                           max_workers=cpu_count, chunksize=1, leave=False)
        for result in pbar:
            df_list.append(result)
        df_dict[split] = pd.DataFrame.from_dict(df_list)
    return df_dict

Define a custom PyTorch `Dataset` for shaping the data from the aformentioned DataFrame dictionary.
- A PyG `Data` object is returned, as it allows for the simplified management of the graph data.
- `torch.from_numpy` is used where possible, as it directly maps memory from a numpy array to a PyTorch tensor without copying.
- `edge_index` has it's axes swapped, as it is stored as `(dest, source)`, and PyG expects edges as `(source, dest)`.
- `edge_index` is transposed, as PyG expects a shape of `(2, num_edges)` instead of `(num_edges, 2)`.
- `torch.contiguous` is called on `edge_index`. PyG recommends this to fully utilize PyTorch [sparse tensors](https://pytorch.org/docs/stable/sparse.html#sparse-coo-docs).
- `config_feat` is multiplied by `100.0`. This helps with training as it results in the magnitude of the graph-level configuration features matching the magnitude of the node-level features.
- `MinMaxScaler` is used on `target`, as we only care about the ranking order. Also, we do not want to target potentially large integers, especially when our loss function, `MSELoss`, requires tensors with a data type of `float32` to function.

In [ ]:
class TileDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        node_feat = torch.from_numpy(row['node_feat'])
        node_opcode = torch.from_numpy(row['node_opcode'].astype(np.int32))
        edge_index = torch.from_numpy(row['edge_index'][:, [1, 0]].T).contiguous()
        config_feat = torch.from_numpy(row['config_feat'] * 100.0)
        config_feat_length = torch.tensor(config_feat.shape[0])
        config_runtime = torch.from_numpy(row['config_runtime'])
        target = config_runtime / torch.from_numpy(row['config_runtime_normalizers'] + 1e-5)

        return Data(node_feat=node_feat, node_opcode=node_opcode, edge_index=edge_index,
                    config_feat=config_feat, target=target, config_runtime=config_runtime,
                   config_feat_length=config_feat_length).to(device)

In [ ]:
tile_df = load_df(path)


tile_df["train"]['node_length'] = tile_df["train"]['node_opcode'].apply(len)
tile_df['train']= tile_df['train'][tile_df["train"]['node_length']<130]
tile_df['train'] = tile_df['train'].drop(columns='node_length')

(1/3) train, Processing:   0%|          | 0/5709 [00:00<?, ?file/s]

(2/3) valid, Processing:   0%|          | 0/676 [00:00<?, ?file/s]

(3/3) test, Processing:   0%|          | 0/844 [00:00<?, ?file/s]

In [ ]:
train_ds = TileDataset(tile_df['train'])
valid_ds = TileDataset(tile_df['valid'])
test_ds = TileDataset(tile_df['test'])

In [ ]:
train_dl = DataLoader(train_ds, shuffle=True)
valid_dl = DataLoader(valid_ds)
test_dl = DataLoader(test_ds)

## The Model

1. All trainable parameters use Kaiming initialization. This is because ReLUs are used throughout. This should help prevent exploding / vanishing gradients, especially at the start of training.

2. Data Preparation:
Wrap all features into a PyTorch Geometric (PyG) Data object for simplified graph data management. Concatenate node features, opcode embeddings and configuration features. An opcode embedding vector of size of `64` was chosen.

3. Feed the concatenated tensor along with the edge index into a PyG `GraphSAGE` model with the following characteristics:
   - Three layers
   - ReLU activation functions (except for the final layer)
   - No normalization
   - Mean aggregation
   - Dropout applied after each aggregation

4. 'Concat pool' the GNN's output. This utilizes PyG's `global_mean_pool` and `global_max_pool` functions to aggregate the output of the GNN, resulting in two seperate graph embeddings. These embeddings are then concatenated:
  - Global Mean Pooling: Averages the features across all nodes, capturing the overall average property of the graph. Get the general characteristics in graphs.
  - Global Max Pooling: Extracts the most significant features from the nodes, focusing on the most prominent characteristics in the graph.


5. The concatenated graph embedding process through three residual layers. Each adding the input back to the output for enhanced information retention.


6. Flatten and normalize the final output to ensure consistency and stability in results.

In [ ]:
class ResidualDenseBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout_rate):
        super(ResidualDenseBlock, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
    def forward(self, x):
        identity = x
        out = self.linear(x)
        out = self.relu(out)
        out = self.dropout(out)
        # Ensure that identity (input) and out have the same dimensions
        if identity.size() == out.size():
            out += identity  # Residual connection
        return out
class TileModel(nn.Module):
    def __init__(self):
        super().__init__()
        emb_dim = 64
        self.embedding = nn.Embedding(120, emb_dim)
        self.gnn = GraphSAGE(in_channels=emb_dim + 140 + 24, hidden_channels=128,
                             num_layers=3, out_channels=64)
        self.compress_layer = nn.Linear(128, 40)
        self.dense_block1 = ResidualDenseBlock(64, 32, p)
        self.dense_block2 = ResidualDenseBlock(32, 32, p)
        self.dense_block3 = ResidualDenseBlock(32, 32, p)
        self.final_linear = nn.Linear(32, 1)
        self.initialize_parameters()
    def initialize_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    init.zeros_(m.bias)
    def forward(self, data):
        num_nodes = data.node_feat.shape[0]
        num_configs = data.config_feat_length.item()
        num_edges = data.edge_index.shape[1]
        x = torch.cat([torch.stack([data.node_feat] * num_configs, dim=1),
                       torch.stack([self.embedding(data.node_opcode)]* num_configs, dim=1),
                       torch.stack([data.config_feat] * num_nodes,dim=0)], dim=-1)
        x = x.permute(1,0,2)
        gnn_out = self.gnn(x, data.edge_index)
        x = torch.cat([global_mean_pool(gnn_out, data.batch), global_max_pool(gnn_out, data.batch)], dim=-1)
        x = self.compress_layer(x.squeeze(1))
        x = torch.cat([data.config_feat, x], dim=-1)
        # residual
        x = self.dense_block1(x)
        x = self.dense_block2(x)
        x = self.dense_block3(x)
        x = self.final_linear(x)
        x = torch.flatten(x)
        x = (x - x.mean()) / (x.std() + 1e-8)
        return x

## Training

### Hyperparameters

These closely match those used in the [paper](https://arxiv.org/abs/2308.13490). Some notable differences are outlined below in [Setting the Scheduler](#Setting-the-Scheduler)

In [ ]:
def listMLE(y_pred, y_true, eps=1e-10, padded_value_indicator=-1):
    """
    ListMLE loss introduced in "Listwise Approach to Learning to Rank - Theory and Algorithm".
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param eps: epsilon value, used for numerical stability
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :return: loss value, a torch.Tensor
    """

    # shuffle for randomised tie resolution
    random_indices = torch.randperm(y_pred.shape[-1])
    y_pred_shuffled = y_pred[random_indices]
    y_true_shuffled = y_true[random_indices]

    y_true_sorted, indices = y_true_shuffled.sort(descending=True, dim=-1)

    mask = y_true_sorted == padded_value_indicator

    preds_sorted_by_true = torch.gather(y_pred_shuffled, dim=0, index=indices)
    preds_sorted_by_true[mask] = float("-inf")

    max_pred_values, _ = preds_sorted_by_true.max(dim=0, keepdim=True)

    preds_sorted_by_true_minus_max = preds_sorted_by_true - max_pred_values

    cumsums = torch.cumsum(preds_sorted_by_true_minus_max.exp().flip(dims=[0]), dim=0).flip(dims=[0])

    observation_loss = torch.log(cumsums + eps) - preds_sorted_by_true_minus_max

    observation_loss[mask] = 0.0
    return torch.mean(torch.sum(observation_loss, dim=0))

In [ ]:
num_epochs = 6
lr = 1e-4
wd = 1e-4
p = 0.1
max_norm = 1e-2
mom = 0.99
pct_start = 0.7
model = TileModel().to(device)
criterion = listMLE
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

### Setting the Scheduler

- A high value of [`max_momentum`](#Hyperparameters) is used. In theory, this should help with training because are only using a single graph configuration at each step without any batching. This results in a noisy set of gradients, as the loss can vary greatly from step to step. Using a high value of momentum smooths this noise inverse to the learning rate (with PyTorch's `OneCycleLR` scheduler), causing the model to converge on a solution in a more consistent manner than it would have otherwise.
- The startup curve for the [One-Cycle Scheduler](https://arxiv.org/abs/1708.07120) (`OneCycleLR`) is set to [`0.7`](#Hyperparameters), which is very high. In theory, this helps as the learning rate should be low and the momentum high as we begin training. This is because it would be easy to overfit with this particular ranking task at the beginning of training. For example, if a series of similar configurations were coincidentally fed into the model at the beginning (which will often happen), the model will overfit to said configurations and have a hard time recovering. If, instead, a low inital learning rate is used, a 'coarse' representation of rankings would be seen by the model at the beginning of training. Only after multiple epochs, when the data has been 'seen' multiple times, should the learning rate be increased, this way, the model is more likely to converge on a better solution.

In [ ]:
scheduler = OneCycleLR(optimizer, max_lr=lr, epochs=num_epochs, steps_per_epoch=len(train_dl),
                       max_momentum=mom, pct_start=pct_start)


### Evaluation Metric

Each configuration is given a score using the `score_preds` function used inside the [`eval`](#The-Training-Loop) function according to the following evaluation metric:

$$1 - \left(\frac{\text{The best runtime of the top-k predictions}}{\text{The best runtime of all configurations}} - 1\right) = 2 - \frac{\min_{i \in K} y_i}{\min_{i \in A} y_i}$$

This formula measures the amount of slowdown incurred for the top-k (`5` in this case) chosen predictions in the range of `0.0-1.0`. At the end of each epoch, the average slowdown is returned so the model's performance can be evaluated on the validation set. Values closer to `1.0` are good, and values closer to `0.0` are bad. If the value falls outside of this range (`>0.0 & <1.0`), something has likely gone wrong during training.

In [ ]:
def score_preds(preds, df):
    score = 0.0
    df_length = len(df)
    for index in range(df_length):
        config_runtime = df.iloc[index]['config_runtime']
        best_pred = min(config_runtime[preds[index]])
        best_targ = min(config_runtime)
        score += 2.0 - best_pred / best_targ
    score /= df_length
    return score

### The Training Loop

In [ ]:
def train():
    model.train()
    pbar = tqdm(train_dl, unit='model', leave=False)
    loss_total = 0.0
    n = 0
    optimizer.zero_grad()
    for data in pbar:
        assert len(data) == 1, 'Batching is unsupported.'
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.target)
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm)
        optimizer.step()
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        loss_total += loss.item()
        n += 1
        pbar.set_description(f'Training: lr: {current_lr:.2e} loss: {(loss_total / n):.4f}')
    return loss_total / n

In [ ]:
def eval():
    preds = []
    model.eval()
    loss_total = 0.0
    pbar = tqdm(valid_dl, desc='Evaluating', unit='model', leave=False)
    for data in pbar:
        out = model(data)
        loss = criterion(out, data.target)
        loss_total += loss.item()
        out = list(out.split(data.config_feat_length.tolist()))
        top_five = [torch.argsort(t.to('cpu').detach())[:5] for t in out]
        preds.extend(top_five)
    loss_total /= len(valid_dl)
    score = score_preds(preds, tile_df['valid'])
    return loss_total, score

In [ ]:
epoch = 1
for epoch in range(num_epochs):
    train_loss = train()
    valid_loss, score = eval()
    display(HTML(f'<div style="font-size: 12px;"><b>Epoch \
                {epoch + 1}/{num_epochs}</b>, Train Loss: {train_loss:.4f} \
                Valid Loss: {valid_loss:.4f}, Score: {score:.4f}</div>'))
    epoch += 1

  0%|          | 0/5705 [00:00<?, ?model/s]

Evaluating:   0%|          | 0/676 [00:00<?, ?model/s]

  0%|          | 0/5705 [00:00<?, ?model/s]

Evaluating:   0%|          | 0/676 [00:00<?, ?model/s]

  0%|          | 0/5705 [00:00<?, ?model/s]

...

Evaluating:   0%|          | 0/676 [00:00<?, ?model/s]

  0%|          | 0/5705 [00:00<?, ?model/s]

Evaluating:   0%|          | 0/676 [00:00<?, ?model/s]

  0%|          | 0/5705 [00:00<?, ?model/s]

Evaluating:   0%|          | 0/676 [00:00<?, ?model/s]

The model typically scores `0.90` and above, meaning that it's predicted configurations for each model in the validation set result in an approximate slowdown of 10% compared to the optimal configurations.

Assuming the test set contains models different to the validation set, it will hopefully perform about the same on the test set.

## Inference

In [ ]:
preds = []
model.eval()
pbar = tqdm(test_dl, desc='Inference', leave=False)
for data in pbar:
    out = model(data)
    out = list(out.split(data.config_feat_length.tolist()))
    top_five = [torch.argsort(t.to('cpu').detach())[:5].tolist() for t in out]
    preds.extend(top_five)

Inference:   0%|          | 0/844 [00:00<?, ?it/s]

In [ ]:
for index, pred in enumerate(preds[:10]):
    file = tile_df['test'].iloc[index]['file']
    print(f'{file}: {pred}')

0070642211d5a98a16b94f4d7df229fe: [1030, 61, 848, 767, 958]
008730b43f100be7c2800d7cb89578a4: [474, 189, 1041, 966, 791]
02c3fb6c0fe263318106bd86942c9ba7: [2156, 2567, 4505, 3499, 3756]
04ae9238c653f8ae08f60f2c03615f0b: [333, 117, 226, 298, 617]
005c91ca7a50fffc663678fd44316f04: [682, 492, 121, 208, 288]
008191e0c67a6e7a62cde1a3e1d66795: [1099, 792, 1077, 292, 978]
038af90a3a43967d62556ed734835c8a: [1548, 7073, 9573, 1121, 4438]
045ce8fc9aee4ddf82c54c0067c8af5e: [961, 2086, 1701, 5894, 4912]
015cd725adcf2df7ec863bd1d9abf058: [1251, 2801, 1072, 3204, 3192]
029880c396f26948e1afbeed8cef9359: [335, 214, 155, 252, 97]


### Submitting the Results

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
for i, file in enumerate(tile_df['test']['file'].values):
    id = f'tile:xla:{file}'
    submission.loc[submission['ID'] == id, 'TopConfigs'] = ';'.join(map(str, preds[i]))
submission.to_csv('./submission.csv', index=False)

In [ ]:
submission

,ID,TopConfigs
0,tile:xla:d6f5f54247bd1e58a10b9e7062c636ab,12;24;23;22;21
1,tile:xla:e3a655daa38e34ec240df959b650ac16,1259;917;1283;396;55
2,tile:xla:f8c2c1a1098b2a361c26df668b286c87,116;41;101;202;44
3,tile:xla:4dd1716853ed46ee4e7d09ede1732de8,8681;9142;4859;1045;5697
4,tile:xla:d0a69155b6340748c36724e4bfc34be3,655;151;650;159;624
...,...,...
889,layout:nlp:random:60880ed76de53f4d7a1b960b24f2...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
890,layout:nlp:random:23559853d9702baaaacbb0c83fd3...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
891,layout:nlp:random:f6c146fc5cf10be4f3accbaca989...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
892,layout:nlp:random:32531d07a084b319dce484f53a4c...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...


- Using'early-joining' the graph configuration features can improve performance according to the [paper](https://arxiv.org/abs/2308.13490).
- Using a ranking loss ([ListMLE](http://icml2008.cs.helsinki.fi/papers/167.pdf))improve performance of the model.